In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = saw
band = [9]
commitnumber = de5d43e
molecule = {'n2o': 3.2e-07}
tsfc = 257


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o n2o band7
  atmpro = mls
  band = 7
  commitnumber = a06b618
  conc = 3.2e-07
  dv = 0.001
  klin = 2.22e-20
  molecule = n2o
  ng_adju = [0, 0]
  ng_refs = [2, 2]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (500, 250)]
  tsfc = 294
  vmax = 1380
  vmin = 1215
  w_diffuse = [(1.8, 1.66), (1.6, 1.8)]
  wgt = [(0.6, 0.5), (0.6, 0.9)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0 -0.016431                  NaN       NaN
0.000312      NaN       NaN                  1.0  0.001637
0.000750      2.0 -0.014944                  2.0  0.002351
0.001052      3.0 -0.011700                  3.0  0.004363
0.001476      4.0 -0.008104                  4.0  0.006420
0.002070      5.0 -0.004174                  5.0  0.008675
0.002904      6.0  0.000116                  6.0  0.011153
0.004074      7.0  0.004761                  7.0  0.013862
0.005714      8.0  0.009797                  8.0  0.016823
0.008015      9.0  0.015244                  9.0  0.020059
0.011243     10.0  0.021118                 10.0  0.023595
0.015771     11.0  0.027436                 11.0  0.027449
0.022122     12.0  0.034215                 12.0  0.031642
0.031031     13.0  0.041428                 13.0  0.036184
0.043528     14.0  0.048107                 14.0  0.040519
0.061057     15.0  0.053251                 15.0  0.044078
0.085645     16.0  0.057585                 16.0  0.047307
0.120136     17.0  0.061846                 17.0  0.050687
0.168516     18.0  0.066039                 18.0  0.054232
0.236378     19.0  0.070183                 19.0  0.057940
0.331549     20.0  0.074368                 20.0  0.061798
0.465100     21.0  0.078206                 21.0  0.065417
0.652400     22.0  0.071907                 22.0  0.062487
0.915100     23.0  0.053797                 23.0  0.051510
1.283650     24.0  0.036096                 24.0  0.039707
1.800600     25.0  0.022900                 25.0  0.029597
2.525700     26.0  0.013582                 26.0  0.020984
3.542800     27.0  0.007307                 27.0  0.013750
4.969550     28.0  0.004014                 28.0  0.008432
6.970850     29.0  0.003025                 29.0  0.005073
9.778100     30.0  0.002615                 30.0  0.002725
13.715850    31.0  0.001991                 31.0  0.000836
19.239350    32.0  0.001131                 32.0 -0.000615
26.987250    33.0  0.000769                 33.0 -0.001216
37.855300    34.0  0.001009                 34.0 -0.001140
53.100050    35.0  0.001106                 35.0 -0.001188
73.887500    36.0  0.000992                 36.0 -0.001574
97.662500    37.0  0.000664                 37.0 -0.002278
121.437500   38.0 -0.000105                 38.0 -0.003329
145.212500   39.0 -0.001153                 39.0 -0.004555
168.987500   40.0 -0.002217                 40.0 -0.005740
192.762500   41.0 -0.003343                 41.0 -0.006890
216.537500   42.0 -0.004571                 42.0 -0.008022
240.312500   43.0 -0.005943                 43.0 -0.009135
264.087500   44.0 -0.007516                 44.0 -0.010236
287.862500   45.0 -0.009212                 45.0 -0.011226
311.637500   46.0 -0.009979                 46.0 -0.011431
335.412500   47.0 -0.009184                 47.0 -0.010474
359.187500   48.0 -0.007719                 48.0 -0.008929
382.962500   49.0 -0.006203                 49.0 -0.007226
406.737500   50.0 -0.004633                 50.0 -0.005411
430.512500   51.0 -0.002986                 51.0 -0.003489
454.287500   52.0 -0.001248                 52.0 -0.001475
478.062500   53.0  0.000582                 53.0  0.000625
501.837500   54.0  0.002495                 54.0  0.002789
525.612500   55.0  0.004478                 55.0  0.004993
549.387500   56.0  0.006503                 56.0  0.007215
573.162500   57.0  0.008539                 57.0  0.009428
596.937500   58.0  0.010548                 58.0  0.011594
620.712500   59.0  0.012493                 59.0  0.013679
644.487500   60.0  0.014342                 60.0  0.015658
668.262500   61.0  0.016076                 61.0  0.017507
692.037500   62.0  0.017696                 62.0  0.019235
715.812500   63.0  0.019214                 63.0  0.020836
739.587500   64.0  0.020653                 64.0  0.022325
76

# Fluxes by Layer

CRD                        CLIRAD                \
                        flug      fldg     fnetg      flug          fldg   
pressure     level                                                         
0.000000e+00 1     -8.678711  0.000000 -8.678711       NaN           NaN   
1.000000e-08 1           NaN       NaN       NaN -8.629700  9.417991e-08   
6.244000e-04 2     -8.678713  0.000001 -8.678712 -8.629700  6.754766e-07   
8.759000e-04 3     -8.678714  0.000001 -8.678713 -8.629700  9.274442e-07   
1.228600e-03 4     -8.678715  0.000002 -8.678713 -8.629700  1.323715e-06   
1.723400e-03 5     -8.678717  0.000003 -8.678714 -8.629701  1.937215e-06   
2.417400e-03 6     -8.678719  0.000005 -8.678714 -8.629701  2.887593e-06   
3.390900e-03 7     -8.678721  0.000007 -8.678714 -8.629701  4.360825e-06   
4.756500e-03 8     -8.678725  0.000011 -8.678713 -8.629701  6.644130e-06   
6.672000e-03 9     -8.678728  0.000017 -8.678711 -8.629701  1.018099e-05   
9.358900e-03 10    -8.678733  0.000027 -8.678706 -8.629700  1.565589e-05   
1.312780e-02 11    -8.678738  0.000041 -8.678697 -8.629698  2.412491e-05   
1.841450e-02 12    -8.678743  0.000063 -8.678680 -8.629694  3.721186e-05   
2.583020e-02 13    -8.678746  0.000097 -8.678650 -8.629686  5.741169e-05   
3.623230e-02 14    -8.678747  0.000148 -8.678599 -8.629673  8.854137e-05   
5.082340e-02 15    -8.678741  0.000226 -8.678515 -8.629650  1.359354e-04   
7.129060e-02 16    -8.678727  0.000340 -8.678386 -8.629614  2.066789e-04   
1.000000e-01 17    -8.678696  0.000506 -8.678190 -8.629558  3.112833e-04   
1.402710e-01 18    -8.678641  0.000745 -8.677895 -8.629470  4.658109e-04   
1.967600e-01 19    -8.678542  0.001088 -8.677454 -8.629335  6.938395e-04   
2.759970e-01 20    -8.678370  0.001575 -8.676795 -8.629127  1.029799e-03   
3.871000e-01 21    -8.678077  0.002261 -8.675816 -8.628808  1.523415e-03   
5.431000e-01 22    -8.677582  0.003212 -8.674371 -8.628319  2.243892e-03   
7.617000e-01 23    -8.676884  0.004375 -8.672509 -8.627660  3.203166e-03   
1.068500e+00 24    -8.676110  0.005556 -8.670554 -8.626913  4.327946e-03   
1.498800e+00 25    -8.675335  0.006621 -8.668714 -8.626131  5.570552e-03   
2.102400e+00 26    -8.674589  0.007512 -8.667077 -8.625347  6.903065e-03   
2.949000e+00 27    -8.673917  0.008202 -8.665714 -8.624613  8.273165e-03   
4.136600e+00 28    -8.673401  0.008714 -8.664687 -8.624010  9.604496e-03   
5.802500e+00 29    -8.673101  0.009207 -8.663895 -8.623603  1.086183e-02   
8.139200e+00 30    -8.673010  0.009953 -8.663057 -8.623431  1.209426e-02   
1.141700e+01 31    -8.673202  0.011160 -8.662042 -8.623634  1.335540e-02   
1.601470e+01 32    -8.673939  0.012981 -8.660958 -8.624561  1.473813e-02   
2.246400e+01 33    -8.675747  0.015653 -8.660094 -8.626809  1.651630e-02   
3.151050e+01 34    -8.679149  0.019880 -8.659270 -8.631076  1.947917e-02   
4.420010e+01 35    -8.684572  0.026820 -8.657752 -8.638230  2.491935e-02   
6.200000e+01 36    -8.693043  0.037623 -8.655420 -8.650054  3.423914e-02   
8.577500e+01 37    -8.705720  0.053095 -8.652626 -8.668691  4.844372e-02   
1.095500e+02 38    -8.719948  0.069193 -8.650755 -8.690374  6.371199e-02   
1.333250e+02 39    -8.736235  0.085184 -8.651051 -8.715333  7.929486e-02   
1.571000e+02 40    -8.755093  0.100793 -8.654300 -8.743723  9.485443e-02   
1.808750e+02 41    -8.776703  0.116158 -8.660545 -8.775437  1.104026e-01   
2.046500e+02 42    -8.801293  0.131332 -8.669961 -8.810404  1.259623e-01   
2.284250e+02 43    -8.829171  0.146338 -8.682833 -8.848590  1.415546e-01   
2.522000e+02 44    -8.860752  0.161183 -8.699569 -8.889948  1.571833e-01   
2.759750e+02 45    -8.896617  0.175880 -8.720737 -8.934445  1.728491e-01   
2.997500e+02 46    -8.937347  0.190666 -8.746681 -8.981911  1.886966e-01   
3.235250e+02 47    -8.981974  0.207188 -8.774786 -9.031202  2.057903e-01   
3.473000e+02 48    -9.028240  0.227589 -8.800651 -9.080593  2.256808e-01   
3.710750e+02 49    -9.074829  0.252438 -8.822391 -9.129109  2.490479e-01 

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')